In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [2]:
AirBnB = pd.read_csv("../original_data/Listings.csv", engine='python', encoding='ISO-8859-1')

In [3]:
AirBnB.head()

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,3705183,39 mÃÂ² Paris (Sacre CÃ âur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


In [4]:
AirBnB.shape

(279712, 33)

In [6]:
AirBnB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279712 entries, 0 to 279711
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   listing_id                   279712 non-null  int64  
 1   name                         279539 non-null  object 
 2   host_id                      279712 non-null  int64  
 3   host_since                   279547 non-null  object 
 4   host_location                278872 non-null  object 
 5   host_response_time           150930 non-null  object 
 6   host_response_rate           150930 non-null  float64
 7   host_acceptance_rate         166625 non-null  float64
 8   host_is_superhost            279547 non-null  object 
 9   host_total_listings_count    279547 non-null  float64
 10  host_has_profile_pic         279547 non-null  object 
 11  host_identity_verified       279547 non-null  object 
 12  neighbourhood                279712 non-null  object 
 13 

`price` is our target column that we will want to try and predict. The rest of the columns are potential features.

Now let's look at the number and % of missing values in our dataset.

In [7]:
missing = pd.concat([AirBnB.isnull().sum(), 100 * AirBnB.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending=False)

,count,%
district,242700,86.767818
host_response_time,128782,46.040928
host_response_rate,128782,46.040928
host_acceptance_rate,113087,40.429799
review_scores_value,91785,32.814109
review_scores_location,91775,32.810534
review_scores_checkin,91771,32.809104
review_scores_accuracy,91713,32.788368
review_scores_communication,91687,32.779073
review_scores_cleanliness,91665,32.771208


Looks like we are missing the majority (over 86%) of the `district` column. Since we have a `city`, a `neighbourhood` and coordinate columns (`latitude` and `longitude`), we should be able to safely drop the `district` column without losing much information. We are missing about a third of all the review related columns and over 40% and 46% of the `host_acceptance_rate` column and `host_response_rate` column (as well as `host_response_time` column) respectively. We will decide what we want to do with these columns/missing values in the future. We are not missing any of our target column, `price`, which is ideal.

In [8]:
AirBnB.district.value_counts()

Manhattan        16553
Brooklyn         14474
Queens            4704
Bronx              992
Staten Island      289
Name: district, dtype: int64

In [9]:
AirBnB.district.value_counts().sum()

37012

In [10]:
AirBnB[AirBnB['city'] == 'New York'].neighbourhood.value_counts()

Williamsburg          2733
Bedford-Stuyvesant    2711
Harlem                1972
Bushwick              1678
Hell's Kitchen        1506
                      ... 
Huguenot                 1
Woodrow                  1
Lighthouse Hill          1
Willowbrook              1
Castle Hill              1
Name: neighbourhood, Length: 220, dtype: int64

In [11]:
AirBnB[AirBnB['city'] == 'New York'].neighbourhood.value_counts().sum()

37012

With all of these districts being only in New York, we can definitely drop this column because it is not providing any extra information that isn't captured by the more specific `neighbourhood` column.

In [12]:
AirBnB = AirBnB.drop('district', axis=1)

Let's have a look at the rows of our dataframe and see if there are any duplicates.

In [13]:
#number of rows duplicated
AirBnB.duplicated().sum()

0

There are no rows that are completely duplicated. What if we ignore the `listing_id` column? 

In [14]:
#rows that are duplicated (except for first instance) when not including listing_id 
dup_rows = AirBnB[AirBnB.drop('listing_id', axis=1).duplicated()]
dup_rows.head()

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
164064,39711694,The New National,304467881,2019-10-25,ZA,NaN,NaN,NaN,f,0.0,...,1,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
66632,47935047,Moderno depto con excelentes amenidades,368595849,2020-09-21,"Mexico City, Mexico City, Mexico",within an hour,1.0,0.93,t,5.0,...,5,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
203497,46753453,Spectacular Apt | Studio in Brooklyn,359229620,2020-07-29,"Houston, Texas, United States",within an hour,1.0,0.84,f,1515.0,...,30,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t
173793,47242698,Well-kept apartment home | 1BR in New York,368944610,2020-09-23,"Houston, Texas, United States",within an hour,1.0,1.00,f,1813.0,...,31,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t
173807,47713309,Well-kept apartment home | 1BR in New York,368944610,2020-09-23,"Houston, Texas, United States",within an hour,1.0,1.00,f,1813.0,...,31,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t


In [15]:
len(dup_rows)

27

In [17]:
#rows that are duplicated on sepcific columns (except for first instance) when not including listing_id 
partial_dup_rows = AirBnB[AirBnB.drop('listing_id', axis=1).duplicated(['name', 'host_id', 'latitude', 'longitude'])]
partial_dup_rows.head()

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
39441,43517742,Super apartment Bastille/Voltaire MOBILITY LEASE,325882341,2020-01-09,FR,within a few hours,0.98,0.80,f,279.0,...,30,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
45214,39211052,Louvre - Richelieu: cosy flat for 2,50978178,2015-12-10,FR,within a few hours,0.73,0.94,f,155.0,...,1,1125,50.0,6.0,9.0,8.0,6.0,9.0,5.0,f
45219,39221386,Louvre - Richelieu: cosy flat for 2,50978178,2015-12-10,FR,within a few hours,0.73,0.94,f,155.0,...,1,1125,72.0,8.0,9.0,8.0,8.0,10.0,8.0,f
48233,41026321,Amazing apartment - 6P/2BR - Canal Saint Martin,319257358,2019-12-20,"Paris, Ile-de-France, France",within an hour,0.99,0.96,f,7.0,...,1,1125,92.0,9.0,9.0,8.0,9.0,10.0,9.0,t
53728,6671646,Central Apartment with 24/7 Concierge (1 guest),29087393,2015-03-10,"Rome, Lazio, Italy",NaN,NaN,NaN,f,9.0,...,1,1125,80.0,8.0,9.0,9.0,9.0,8.0,8.0,f


In [18]:
len(partial_dup_rows)

356

We can see that only 27 rows have the same information when not including the `listing_id` column. However, there are 356 columns where the name, host, latitude and longitude are the same. 

Since all of these rows have different listing ids, however, it is possible that they are listings that were taken down and put back up at a later date for whatever reason, and therefore given a new listing id. This could be from a customer making changes to the property and wanting to change the price while also not wanting old reviews to be associated with the listing. 

Therefore, I am  going to keep these rows since it won't really impact our models and they might be able to learn something from this customer behavior. 

Let's tackle the numeric features of our dataset.

In [78]:
AirBnB.describe().T

,count,mean,std,min,25%,50%,75%,max
listing_id,279712.0,2.638196e+07,1.442576e+07,2577.00000,1.384462e+07,2.767098e+07,3.978485e+07,4.834353e+07
host_id,279712.0,1.081658e+08,1.108570e+08,1822.00000,1.720656e+07,5.826911e+07,1.832853e+08,3.901874e+08
host_response_rate,150930.0,8.659386e-01,2.837443e-01,0.00000,9.000000e-01,1.000000e+00,1.000000e+00,1.000000e+00
host_acceptance_rate,166625.0,8.271684e-01,2.892025e-01,0.00000,7.800000e-01,9.800000e-01,1.000000e+00,1.000000e+00
host_total_listings_count,279547.0,2.458161e+01,2.840411e+02,0.00000,1.000000e+00,1.000000e+00,4.000000e+00,7.235000e+03
latitude,279712.0,1.876186e+01,3.256034e+01,-34.26440,-2.296439e+01,4.071079e+01,4.190861e+01,4.890491e+01
longitude,279712.0,1.259508e+01,7.308131e+01,-99.33963,-4.319804e+01,2.382780e+00,2.898673e+01,1.513398e+02
accommodates,279712.0,3.288736e+00,2.133379e+00,0.00000,2.000000e+00,2.000000e+00,4.000000e+00,1.600000e+01
bedrooms,250277.0,1.515509e+00,1.153080e+00,1.00000,1.000000e+00,1.000000e+00,2.000000e+00,5.000000e+01
price,279712.0,6.087927e+02,3.441827e+03,0.00000,7.500000e+01,1.500000e+02,4.740000e+02,6.252160e+05


Let's have a look at the Categorical features now.

In [62]:
# number of unique values in categorical columns
unique_cat_vals = AirBnB.select_dtypes('object').nunique()
unique_cat_vals

name                      265861
host_since                  4240
host_location               7159
host_response_time             4
host_is_superhost              2
host_has_profile_pic           2
host_identity_verified         2
neighbourhood                660
city                          10
property_type                144
room_type                      4
amenities                 245003
instant_bookable               2
dtype: int64

In [64]:
#number of unique values in categorical columns with less that 10 unique values 
small_unique_cat_vals = unique_cat_vals[np.where(unique_cat_vals < 10, True, False)]
small_unique_cat_vals

host_response_time        4
host_is_superhost         2
host_has_profile_pic      2
host_identity_verified    2
room_type                 4
instant_bookable          2
dtype: int64

In [67]:
#one-hot-encode the categorical columns with less than 10 unique values
dummy_AirBnB = pd.get_dummies(AirBnB, columns=small_unique_cat_vals.index, dummy_na=True, drop_first=True)

In [74]:
dummy_AirBnB.shape

(279712, 42)

In [72]:
dummy_AirBnB.select_dtypes(np.uint8).head()

,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan,host_is_superhost_t,host_is_superhost_nan,host_has_profile_pic_t,host_has_profile_pic_nan,host_identity_verified_t,host_identity_verified_nan,room_type_Hotel room,room_type_Private room,room_type_Shared room,room_type_nan,instant_bookable_t,instant_bookable_nan
0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0
2,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0


In [73]:
dummy_AirBnB.select_dtypes(np.uint8).shape

(279712, 16)

We've created 16 encoded columns and dropped the 6 original columns in our dummy DataFrame. Some models will want to use the dummy DataFrame while others will want to use the original one. This will depend on the models we choose (e.g. tree based models will not want to use the dummy DataFrame).

In [76]:
dummy_AirBnB.select_dtypes('object').nunique()

name             265861
host_since         4240
host_location      7159
neighbourhood       660
city                 10
property_type       144
amenities        245003
dtype: int64